In [ ]:
from matplotlib import pyplot as pl
%matplotlib inline
import mpld3
import numpy as np
mpld3.enable_notebook()

# Demo of image normalization and corrections when doing fft to/ftom uv

[Colab Link](https://colab.research.google.com/github/casangi/astroviper/blob/main/docs/core_tutorials/imaging/demo_fft_ifft_handling.ipynb)


Description of notebook

In dealing with gridded visibilities and images and moving from uv domain to lm domain we have to do fft's. But to get the correct values and reduce errors there are certain steps that are needed, that we do not usually discuss in theory but has to happen in practice.

- In doing Fourier Transform there is a factor N which is the length of the data sample that is transformed that gets applied in the results. This has to be normalized out
- Using a convolution function in gridding means there is the FT of the convultion function that is applied to the resultant image. Thus it has be be removed to get correct flux density for e.g
-  We may want to pad an image before FFT (or oversample a uv grid) to reduce effects of aliasing and Gibbs phenomenon ringing. We therefore have to add padding and remove it before presenting the final image. 

## Assumptions





## Install AstroVIPER



In [ ]:
from importlib.metadata import version
import os

try:
    os.system("pip install --upgrade astroviper")

    import astroviper

    print("Using astroviper version", version("astroviper"))

except ImportError as exc:
    print(f"Could not import astroviper: {exc}")

In [ ]:
from astroviper.core.imaging.imaging_utils.standard_image_grid_preparation import (
    make_image_xds,
    remove_padding,
    apply_pb,
    make_empty_padded_uv_image,
    fft_to_uv,
    ifft_to_lm,
    correct_fft_to_uv,
    correct_ifft_to_lm,
    grid2xradio_spheroid_ms4,
)


## Helper function

The `make_image_xds` helper used below creates a blank sky-image dataset from basic imaging parameters and is provided by `standard_image_grid_preparation`.


Lets generate a ms4 dataArray with a few point sources in it

sources will have the position and flux of source, npix is the size of the image you may wish to make, cell the cellsize determined from max uv and ms4 will be an xradio msv4 dataArray Here we are putting 4 point sources of 1 Jy visibilities


In [ ]:
from astroviper.core.imaging.imaging_utils.standard_gridding_example import generate_ms4_with_point_sources
nsources = 4
sources, npix, cell, ms4 = generate_ms4_with_point_sources(nsources, np.ones(nsources))

Lets collect some meta data to make an image which we will use to grid the visibilities on

In [ ]:
pc= ms4.field_and_source_base_xds.FIELD_PHASE_CENTER_DIRECTION.data[0]
freqinc=ms4.frequency.attrs['channel_width']['data']
freq=ms4.frequency.data[0]
cellsize=[-cell.to_value('rad'),cell.to_value('rad')]
npixels=[int(npix*1.2), int(npix*1.2)]

Make a blank image, then use that to make an image with padding and with uv coordinates and use it to grid the visibility from the ms4

In [ ]:
tempim=make_image_xds(imsize=npixels, cell=cellsize, freq=freq, freqinc=freqinc, phasecent=pc, which="RESIDUAL")
grid=make_empty_padded_uv_image(tempim, image_size=np.array([240, 240]), uv_data_array="VISIBILITY_RESIDUAL", insert_res_or_mod=True)

In [ ]:
# Standard image grid preparation utilities were imported in the cell above.


In [ ]:
ms4["VISIBILITY_RESIDUAL"]=ms4["VISIBILITY"]
grid2xradio_spheroid_ms4(ms4, grid, data_var="VISIBILITY_RESIDUAL")

FFT the gridded visibility and display what the image looks like. Can be seen the tapering due to the convolution function used.

In [ ]:
ifft_to_lm(grid)

In [ ]:
pl.imshow(grid.RESIDUAL.data[0,0,0,])

Correct the convolution function effect, sum of weights of the visibilities and FFT scale factor.

In [ ]:
correct_ifft_to_lm(grid, doSpheroidCorr=True)

In [ ]:
pl.imshow(grid.RESIDUAL.data[0,0,0,])

Remove the padding of 20% that was added

In [ ]:
b=remove_padding(grid, image_size=[200,200])

In [ ]:
pl.imshow(b.RESIDUAL.data[0,0,0,])